In [ ]:
# Install required packages
!pip install transformers flask flask-cors pyngrok --quiet
!pip install sentence-transformers scikit-learn --quiet
!pip install sacremoses --quiet

# Load Models

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT-Large")

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

biobert = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb', device=device)

# Document Embeddings & Retriver

In [ ]:
docs = []
doc_embedings = []

In [ ]:
def add_doc(text):
    # Embed the incoming doc using BioBERT
    embedding = biobert.encode([text], convert_to_tensor=True)[0].cpu().numpy()

    # Store in memory
    docs.append(text)
    doc_embeddings.append(embedding)

    print(f"✅ Added document. Total docs: {len(docs)}")

In [ ]:
def find_most_similar_doc(query, top_k=1):
    if not doc_embeddings:
        return "❌ No documents in memory."

    # Embed the user query
    query_embedding = biobert.encode([query], convert_to_tensor=True)[0].cpu().numpy()

    # Compute cosine similarities
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # Get the top-k most similar docs
    top_indices = similarities.argsort()[-top_k:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "doc": docs[idx],
            "similarity": float(similarities[idx])
        })

    return results

# API

In [ ]:
!pip install flask flask-cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/embed-doc', methods=['POST'])
def embed_doc():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "Missing 'text' field."}), 400

    add_doc(text)
    return jsonify({"message": "✅ Document embedded.", "total_docs": len(docs)})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "Missing 'query' field."}), 400

    results = find_most_similar_doc(query, top_k=1)
    return jsonify(results)

In [ ]:
!pip install pyngrok

In [ ]:
token = input('Enter ngrok auth token: ')

In [ ]:
from pyngrok import ngrok

# Replace with your token from ngrok.com
ngrok.set_auth_token(token)

In [ ]:
# Run Flask app in background
import threading

def run_flask():
    from flask_cors import CORS
    CORS(app)
    app.run(port=5000)

thread = threading.Thread(target=run_flask)
thread.start()

# Start ngrok tunnel
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)